
<a href="http://landlab.github.io"><img style="float: left; width: 300px;" src="https://landlab.csdms.io/_static/landlab_logo.png"></a>

# 2D Surface Water Flow component: Modeling water flow over complex terrain

<hr>
<small>For more Landlab tutorials, click here: <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html</a></small>
<hr>


## Overview

This notebook demonstrates the usage of the `river flow dynamics` Landlab component with a real-world digital elevation model (DEM). The component runs a semi-implicit, semi-Lagrangian finite-volume approximation to the depth-averaged 2D shallow-water equations of Casulli and Cheng (1992) and related work.

### Theory

The depth-averaged 2D shallow-water equations are the simplification of the Navier-Stokes equations, which correspond to the balance of momentum and mass in the fluid. It is possible to simplify these equations by assuming a well-mixed water column and a small water depth to width ratio, where a vertical integration results in depth-averaged equations. These require boundary conditions at the top and bottom of the water column, which are provided by the wind stress and the Manning-Chezy formula, respectively:

$$
\frac{\partial U}{\partial t}
+ U\frac{\partial U}{\partial x} + V\frac{\partial U}{\partial y}
= 
- g\frac{\partial \eta}{\partial x}
+ \epsilon\left(\frac{\partial^2 U}{\partial x^2} + \frac{\partial^2 U}{\partial y^2}\right)
+ \frac{\gamma_T(U_a - U)}{H} - g\frac{\sqrt{U^2 + V^2}}{Cz^2}U + \mathbf{f}V
$$

$$
\frac{\partial V}{\partial t}
+ U\frac{\partial V}{\partial x} + V\frac{\partial V}{\partial y}
= 
- g\frac{\partial \eta}{\partial y}
+ \epsilon\left(\frac{\partial^2 V}{\partial x^2} + \frac{\partial^2 V}{\partial y^2}\right)
+ \frac{\gamma_T(V_a - V)}{H} - g\frac{\sqrt{U^2 + V^2}}{Cz^2}V + \mathbf{f}U
$$

$$
\frac{\partial \eta}{\partial t}
+ \frac{\partial (HU)}{\partial x} + \frac{\partial (HV)}{\partial y}
= 0
$$

where $U$ is the water velocity in the $x$-direction, $V$ is the water velocity in the $y$-direction, $H$ is the water depth, $\eta$ is the water surface elevation, $Cz$ is the Chezy friction coefficient, and $t$ is time. For the constants $g$ is the gravity acceleration, $\epsilon$ is the horizontal eddy viscosity, $\mathbf{f}$ is the Coriolis parameter, $\gamma_T$ is the wind stress coefficient, and $U_a$ and $V_a$ are the prescribed wind velocities.

### Numerical representation

A semi-implicit, semi-Lagrangian, finite volume numerical approximation represents the depth averaged, 2D shallow-water equations described before. The water surface elevation, $\eta$, is defined at the center of each computational volume (nodes). Water depth, $H$, and velocity components, $U$ and $V$, are defined at the midpoint of volume faces (links). The finite volume structure provides a control volume representation that is inherently mass conservative.

The combination of a semi-implciit water surface elevation solution and a semi-Lagrangian representation of advection provides the advantages of a stable solution and of time steps that exceed the CFL criterion. In the semi-implicit process, $\eta$ in the momentum equations, and the velocity divergence in the continuity equation, are treated implicitly. The advective terms in the momentum equations, are discretized explicitly. See the cited literature for more details.

### The component

Import the needed libraries:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from tqdm import trange

from landlab import RasterModelGrid
from landlab.components import RiverFlowDynamics
from landlab.io import esri_ascii
from landlab.plot.imshow import imshow_grid

## Information about the component

Using the class name as argument for the `help` function returns descriptions of the various methods and parameters.

In [ ]:
help(RiverFlowDynamics)

-- --
## Example 2: Modeling water flow over complex terrain

In this example, we will import a digital elevation model (DEM) for a side-channel of the Kootenai River, Idaho, US. We'll demonstrate how the `RiverFlowDynamics` component can simulate water flow over real-world terrain.

### Loading the Digital Elevation Model

We'll load the DEM from an ASCII file and use it as our topographic elevation field.

In [ ]:
# Getting the grid and some parameters
asc_file = "DEM_kootenai_37x50_1x1.asc"
with open(asc_file) as fp:
    grid = esri_ascii.load(fp, name="topographic__elevation")
te = grid.at_node["topographic__elevation"]

### Define Simulation Parameters

We specify basic parameters such as Manning's roughness coefficient, time step duration, and number of time steps.

In [ ]:
# Basic parameters
mannings_n = 0.012  # Manning's roughness coefficient, [s/m^(1/3)]

# Simulation parameters
n_timesteps = 75  # Number of timesteps
dt = 1.0  # Timestep duration, [s]

# Display parameters
display_animation_freq = 5  # How often to display animation frames

### Visualize Initial Topography

Let's examine the topography to get a better understanding of the terrain we're working with.

In [ ]:
# Showing the topography with enhanced visualization
plt.figure(figsize=(12, 8))
im = imshow_grid(grid, "topographic__elevation", cmap="terrain")
plt.colorbar(im, label="Elevation (m)")
plt.title("Kootenai River Side-Channel Topography", fontsize=14)
plt.show()

### Set Initial Conditions

Our side-channel is empty at the beginning of the simulation, so we create the fields for water depth, velocity, and water surface elevation.

In [ ]:
# We establish the initial conditions, which represent an empty channel
h = grid.add_zeros("surface_water__depth", at="node")

# Water velocity is zero in everywhere since there is no water yet
vel = grid.add_zeros("surface_water__velocity", at="link")

# Calculating the initial water surface elevation from water depth and topographic elevation
wse = grid.add_field("surface_water__elevation", te, at="node")

### Set Boundary Conditions

We specify the nodes and links where water enters the domain, along with the water depth and velocity values at these entry points. In this case, water will flow from right to left.

In [ ]:
# We set fixed boundary conditions, specifying the nodes and links in which the water is flowing into the grid
fixed_entry_nodes = grid.nodes_at_right_edge
fixed_entry_links = grid.links_at_node[fixed_entry_nodes][:, 2]

# We set the fixed values in the entry nodes/links
entry_nodes_h_values = np.array(
    [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.04998779,
        0.05999756,
        0.03997803,
        0.0,
        0.0,
        0.0,
        0.05999756,
        0.10998535,
        0.12994385,
        0.09997559,
        0.15997314,
        0.23999023,
        0.30999756,
        0.36999512,
        0.45996094,
        0.50994873,
        0.54998779,
        0.59997559,
        0.63995361,
        0.65997314,
        0.65997314,
        0.60998535,
        0.5,
        0.13995361,
        0.0,
    ]
)
entry_links_vel_values = np.array(
    [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        -2.58638018,
        -2.58638018,
        -2.58638018,
        0.0,
        0.0,
        0.0,
        -2.58638018,
        -2.58638018,
        -2.58638018,
        -2.58638018,
        -2.58638018,
        -2.58638018,
        -2.58638018,
        -2.58638018,
        -2.58638018,
        -2.58638018,
        -2.58638018,
        -2.58638018,
        -2.58638018,
        -2.58638018,
        -2.58638018,
        -2.58638018,
        -2.58638018,
        -2.58638018,
        0.0,
    ]
)

### Visualize Entry Cross-section

Let's plot the entry boundary condition in the cross-section with enhanced visualization to better understand the water inflow.

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(
    grid.y_of_node[fixed_entry_nodes], 
    entry_nodes_h_values + te[fixed_entry_nodes],
    'b-', linewidth=2, label="Water Surface"
)
plt.plot(
    grid.y_of_node[grid.nodes_at_right_edge], 
    te[grid.nodes_at_right_edge],
    'brown', linewidth=2, label="Channel Bottom"
)
plt.fill_between(
    grid.y_of_node[fixed_entry_nodes],
    te[fixed_entry_nodes],
    entry_nodes_h_values + te[fixed_entry_nodes],
    color='blue', alpha=0.3
)
plt.title("Entry Cross-section at Right Boundary", fontsize=14)
plt.xlabel("Distance along cross-section [m]", fontsize=12)
plt.ylabel("Elevation [m]", fontsize=12)
plt.grid(True)
plt.legend()
plt.show()

### Initialize the RiverFlowDynamics Component

Now we create the component by passing the arguments defined previously.

In [ ]:
# Initialize the model with our parameters and boundary conditions
rfd = RiverFlowDynamics(
    grid,
    dt=dt,
    mannings_n=mannings_n,
    fixed_entry_nodes=fixed_entry_nodes,
    fixed_entry_links=fixed_entry_links,
    entry_nodes_h_values=entry_nodes_h_values,
    entry_links_vel_values=entry_links_vel_values,
)

### Run the Simulation

We run the simulation for 75 time steps (75 seconds) with improved visualization of the water propagation through the channel.

In [ ]:
for timestep in trange(n_timesteps):
    rfd.run_one_step()

    if timestep % display_animation_freq == 0:
        clear_output(wait=True)  # Clear the previous image
        plt.figure(figsize=(12, 8))
        im = imshow_grid(grid, "surface_water__depth", cmap="Blues")
        plt.colorbar(im, label="Water Depth (m)")
        plt.title(f"Water Depth at Time Step {timestep+1} (Time: {(timestep+1)*dt}s)")
        plt.show()

### Visualize Final Results

Let's examine the water depth and water surface elevation at the end of the simulation with enhanced visualization.

In [ ]:
# Final water depth
plt.figure(figsize=(12, 8))
im = imshow_grid(grid, "surface_water__depth", cmap="Blues")
plt.colorbar(im, label="Water Depth (m)")
plt.title(f"Final Water Depth after {n_timesteps} Time Steps ({n_timesteps*dt}s)")
plt.show()

In [ ]:
# Final water surface elevation
plt.figure(figsize=(12, 8))
im = imshow_grid(grid, "surface_water__elevation", cmap="terrain")
plt.colorbar(im, label="Water Surface Elevation (m)")
plt.title(f"Final Water Surface Elevation after {n_timesteps} Time Steps ({n_timesteps*dt}s)")
plt.show()

### Visualize Flow Velocity Magnitude

Let's calculate and visualize the flow velocity magnitude across the domain to better understand the flow dynamics.

In [ ]:
# Extract velocity components
links_at_node = grid.links_at_node
velocity_at_links = grid.at_link["surface_water__velocity"]

# Calculate velocity magnitude at nodes (approximate)
velocity_magnitude = np.zeros(grid.number_of_nodes)
for i in range(grid.number_of_nodes):
    # Get valid links (those that are not -1)
    valid_links = links_at_node[i][links_at_node[i] >= 0]
    if len(valid_links) > 0:
        # Take the maximum velocity of connected links as an approximation
        velocity_magnitude[i] = np.max(np.abs(velocity_at_links[valid_links]))

# Add velocity magnitude field to grid
grid.at_node["velocity_magnitude"] = velocity_magnitude

# Plot velocity magnitude
plt.figure(figsize=(12, 8))
im = imshow_grid(grid, "velocity_magnitude", cmap="plasma")
plt.colorbar(im, label="Velocity Magnitude (m/s)")
plt.title(f"Flow Velocity Magnitude after {n_timesteps} Time Steps")
plt.show()

### Analyze Cross-sections

Let's examine the water depth and flow along cross-sections of the domain to better understand the simulation results.

In [ ]:
# Find the middle of the domain for cross-sections
mid_row = grid.shape[0] // 2
mid_col = grid.shape[1] // 2

# Extract cross-section through the middle row (longitudinal profile)
row_nodes = np.arange(mid_row * grid.shape[1], (mid_row + 1) * grid.shape[1], dtype=int)

# Plot longitudinal cross-section
plt.figure(figsize=(14, 6))

# Plot terrain
plt.plot(grid.x_of_node[row_nodes], grid.at_node['topographic__elevation'][row_nodes], 
         'k-', linewidth=2, label='Terrain')

# Plot water surface
plt.plot(grid.x_of_node[row_nodes], grid.at_node['surface_water__elevation'][row_nodes], 
         'b-', linewidth=2, label='Water Surface')

# Fill between terrain and water surface where there's water
water_indices = grid.at_node['surface_water__depth'][row_nodes] > 0
plt.fill_between(grid.x_of_node[row_nodes][water_indices], 
                 grid.at_node['topographic__elevation'][row_nodes][water_indices],
                 grid.at_node['surface_water__elevation'][row_nodes][water_indices],
                 color='blue', alpha=0.3)

plt.grid(True)
plt.xlabel('Distance Along Channel (m)', fontsize=12)
plt.ylabel('Elevation (m)', fontsize=12)
plt.title('Longitudinal Profile (East-West) through Domain Center', fontsize=14)
plt.legend()
plt.show()

## Interpretation of Results

The simulation results demonstrate how the `RiverFlowDynamics` component can effectively model water flow over real-world topography. Key observations from this simulation include:

1. **Channel-following flow**: The water naturally follows the lowest elevations in the DEM, demonstrating how topography controls flow paths in natural systems.

2. **Flow acceleration and deceleration**: The velocity magnitude visualization shows how water accelerates in steep, narrow sections and slows down in flatter, wider areas, reflecting real-world fluid dynamics.

3. **Depth variation**: Water depth varies across the channel, with deeper sections in topographic lows and shallower flow over higher terrain, creating a realistic representation of natural channel hydraulics.

4. **Flow evolution**: The animation shows the progressive filling of the channel over time, starting from the inflow boundary and gradually propagating downstream as would occur in nature.

This simulation provides valuable insights into how water interacts with natural terrain and demonstrates the capability of the `RiverFlowDynamics` component to model complex hydrodynamic processes over real-world topography.

-- --
### And that's it! 

Nice work completing this tutorial. You know now how to use the `RiverFlowDynamics` Landlab component to simulate water flow over real-world DEMs :)

-- --



### Click here for more <a href="https://landlab.csdms.io/tutorials/">Landlab tutorials</a>